In [13]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import lstm_models as models

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, CuDNNLSTM, Conv1D, MaxPooling1D, Activation, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
import re
import zipfile

In [2]:
max_features = 10000
X_train = np.load('../sequence/x_train_word_{}.npy'.format(max_features))
X_dev = np.load('../sequence/x_dev_word_{}.npy'.format(max_features))
X_test = np.load('../sequence/x_test_word_{}.npy'.format(max_features))
Y_train = np.load('../sequence/y_train.npy')
Y_dev = np.load('../sequence/y_dev.npy')
Y_test = np.load('../sequence/y_test.npy')

In [9]:
print('The shape of X_train: {}'.format(X_train.shape))
print('The shape of X_dev: {}'.format(X_dev.shape))
print('The shape of X_test: {}'.format(X_test.shape))

The shape of X_train: (1024000, 99)
The shape of X_dev: (256000, 99)
The shape of X_test: (320000, 99)


In [4]:
print('The shape of Y_train: {}'.format(Y_train.shape))
print('The shape of Y_dev: {}'.format(Y_dev.shape))
print('The shape of Y_test: {}'.format(Y_test.shape))

The shape of Y_train: (1024000, 2)
The shape of Y_dev: (256000, 2)
The shape of Y_test: (320000, 2)


In [22]:
# Embedding
embedding_size = 128
input_size = X_train.shape[1]

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 100

In [23]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=input_size))
model.add(Dropout(0.5))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(CuDNNLSTM(lstm_output_size))
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
batch_size = 128
epochs = 2

history = model.fit(X_train, Y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_dev, Y_dev))

Train on 1024000 samples, validate on 256000 samples
Epoch 1/2
1024000/1024000 [==============================] - 131s 128us/step - loss: 0.5710 - acc: 0.6942 - val_loss: 0.5550 - val_acc: 0.7060
Epoch 2/2
1024000/1024000 [==============================] - 129s 126us/step - loss: 0.5497 - acc: 0.7107 - val_loss: 0.5495 - val_acc: 0.7100


In [26]:
score, acc = model.evaluate(X_test, Y_test, batch_size = batch_size)
print("score: %.8f" % (score))
print("acc: %.8f" % (acc))

320000/320000 [==============================] - 10s 30us/step
score: 0.55094036
acc: 0.70935781
